<a href="https://colab.research.google.com/github/splenwilz/golf_swing_analysis/blob/main/Golf_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load dataset with pyomeca

In [ ]:
import pyomeca
data = pyomeca.C3D("golf_swing.c3d")

##Preprocessing the data

In [ ]:
data_processed = data.meca.filter_butter(
    f_cutoff=10, # cut-off frequency in Hz
    order=4, # order of the filter
    direction="lowpass", # filter direction
)

# Calculating Joint Angles
angles = pyomeca.to_radians(data_processed.meca.joint_angles())

# Calculate the joint velocities and accelerations using Pyomeca
velocities = data_processed.meca.velocity(angles)
accelerations = data_processed.meca.acceleration(angles, velocities)

##Create a Musculoskeletal Model

In [ ]:
import biorbd

model = biorbd.Model.from_path('model.bioMod')

#Define the markers used in the motion capture data
markers = pyomeca.Markers.from_c3d("golf_swing.c3d")
marker_names = markers.names.tolist()

#Set the position of the markers in the model
model_markers = pyomeca.Markers.from_biorbd(model, marker_names)
model_markers.set_current_frame(markers[:, 0, :])


##Calculate Muscle Forces, Joint Torques, and Power Output

In [ ]:
#Calculate the muscle activations using Biorbd.
activations = biorbd.VecBiorbdMuscle(model.nbMuscleTotal())

#Calculate the muscle forces and joint torques using Biorbd
muscle_forces = model.muscleForces(activations, angles, velocities)
joint_torques = model.torque(angles, velocities, activations)

#Calculate the power output of the movement using Biorbd
power_output = joint_torques @ velocities.T


# These lines of code use the previously calculated joint angles, velocities, 
# and muscle activations to estimate the muscle forces, joint torques, and 
# power output of the golf swing. Note that the power output is calculated 
# as the dot product of the joint torques and velocities.



##Define the Optimization Problem

In [ ]:
import casadi as ca

#Define the decision variables for the optimization problem. In this case, the decision variables are the muscle activations.
nlp = ca.tools.nlp(N=1, h=0.01)
activations = nlp.add_variable(name="activations", shape=(model.nbMuscleTotal(),))


#Define the objective function
objective = ca.sumsqr(muscle_forces)
nlp.add_objective(objective)

#Define the constraints on the decision variables.
nlp.add_constraint(0.01 <= activations)
nlp.add_constraint(activations <= 1)


#Define the constraints on the state variables.
nlp.add_constraint(angles[:, -1] - angles[:, 0], lower=-0.1, upper=0.1)
nlp.add_constraint(velocities[:, -1], lower=0, upper=0)
nlp.add_constraint(accelerations[:, -1], lower=0, upper=0)

#Define the constraints on the control variables
nlp.add_constraint(muscle_forces, lower=0, upper=1000)
nlp.add_constraint(joint_torques, lower=-1000, upper=1000)


#Define the constraints on the initial and final values of the state variables.
nlp.add_constraint(angles[:, 0], equal=data_processed["markers"][:, 0])
nlp.add_constraint(velocities[:, 0], equal=data_processed["qdot"][:, 0])
nlp.add_constraint(angles[:, -1], equal=data_processed["markers"][:, -1])
nlp.add_constraint(velocities[:, -1], equal=data_processed["qdot"][:, -1])


##Solve the Optimization Problem

In [ ]:
options = {"ipopt.tol": 1e-3, "ipopt.print_level": 0}
solver = nlp.solver("ipopt", options)
init_guess = {"activations": np.zeros((model.nbMuscleTotal(),))}
sol = solver.solve(init_guess)


##Visualize the Results

In [ ]:
time = np.linspace(0, data["n_frames"] / data["frame_rate"], data["n_frames"])
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
for i, ax in enumerate(axs):
    ax.plot(time, angles[i, :])
    ax.set_ylabel(f"Joint {i + 1} angle (rad)")
axs[-1].set_xlabel("Time (s)")


# This code creates a plot with three subplots, one for each joint angle. 
# The plot shows how the joint angles change over time during the golf swing

In [ ]:
#We can also plot the joint velocities and accelerations using similar code

In [ ]:
time = np.linspace(0, data["n_frames"] / data["frame_rate"], data["n_frames"])
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
for i, ax in enumerate(axs):
    ax.plot(time, velocities[i, :])
    ax.set_ylabel(f"Joint {i + 1} velocity (rad/s)")
axs[-1].set_xlabel("Time (s)")

# This code creates a plot with three subplots, one for each joint velocity. 
# The plot shows how the joint velocities change over time during the golf swing.

In [ ]:
#Similarly, to plot the joint accelerations,
time = np.linspace(0, data["n_frames"] / data["frame_rate"], data["n_frames"])
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
for i, ax in enumerate(axs):
    ax.plot(time, accelerations[i, :])
    ax.set_ylabel(f"Joint {i + 1} acceleration (rad/s^2)")
axs[-1].set_xlabel("Time (s)")


# This code creates a plot with three subplots, one for each joint acceleration. 
# The plot shows how the joint accelerations change over time during the golf swing.

In [ ]:
#We can also plot the muscle forces, joint torques, and power output using similar code
time = np.linspace(0, data["n_frames"] / data["frame_rate"], data["n_frames"])
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
axs[0].plot(time, muscle_forces.T)
axs[0].set_ylabel("Muscle Forces (N)")
axs[1].plot(time, joint_torques.T)
axs[1].set_ylabel("Joint Torques (N.m)")
axs[2].plot(time, power_output.T)
axs[2].set_ylabel("Power Output (W)")
axs[-1].set_xlabel("Time (s)")


# This code creates a plot with three subplots, one for each variable. 
# The plot shows how the muscle forces, joint torques, and power output 
# change over time during the golf swing.